# Submitting Multiple Workflows 

### Import Helper Functions

In [ ]:
from atsclientslibraries.workflow_library import workflowLibrary
from atsclientslibraries.workflow_schedules import workflowSchedules
from atsclientslibraries.workflows import workflows
from atsclientslibraries.workflow_invoke import workflowInvoke

### Define Global Configurations

In [ ]:
ENV = "dev" # Change to your Default Environment

## The Team Name configured in the Data Lake platform
# This is the same value passed in the 'data_lake_parameters' team of ddk.json file
TEAM_NAME = "<INSERT TEAM NAME>" 

#ENTER YOUR CUSTOMER ID SET IN THE 'Getting_Started_With_AMC_Quickstart' NOTEBOOK 
customer_id = "testdemocustomer"

### Define a List of AMC Workflows to Submit

In [ ]:
workflow_list = [
     {
         "defaultSchedule": {
             "automaticDeploySchedule": False,
             "Description": "Runs the exposure_group_analysis workflow at 11:00 GMT / 9:00am EST daily. ",
             "Input": {
                 "payload": {
                     "timeWindowEnd": "today(-2)",
                     "timeWindowStart": "today(-3)",
                     "timeWindowType": "EXPLICIT",
                     "workflow_executed_date": "now()"
                 }
             },
             "Name": "exposure_group_analysis_daily",
             "ScheduleExpression": "custom(D * 11)",
             "State": "ENABLED"
         },
         "filteredMetricsDiscriminatorColumn": "filtered",
         "sqlQuery": "WITH IMP_CLICKSSEG AS  ( SELECT A.ADVERTISER, A.behavior_segment_name AS SEGMENT, SUM(A.total_cost/100000) AS impression_cost, behavior_segment_matched AS MATCHED_SEGMENT, SUM(A.IMPRESSIONS) AS IMPRESSIONS, SUM(B.clicks) as CLICKS, COUNT(DISTINCT A.USER_ID) AS REACH FROM display_impressions_by_user_segments A LEFT JOIN DISPLAY_CLICKS B ON A.request_tag = B.request_tag GROUP BY A.ADVERTISER, A.behavior_segment_name, A.behavior_segment_matched), USR_SEG_IMP as ( select user_id, advertiser_id, behavior_segment_name, behavior_segment_matched, sum(total_cost)/100000 as impression_cost from display_impressions_by_user_segments group by user_id, advertiser_id, behavior_segment_name, behavior_segment_matched ), CONV_SEG AS ( SELECT A.ADVERTISER, A.tracked_asin, B.behavior_segment_name AS SEGMENT, B.behavior_segment_matched AS MATCHED_SEGMENT, SUM(A.CONVERSIONS) AS PURCHASES, sum(A.product_sales) as sales_tracked, sum(A.total_product_sales) as sales_tracked_brand, ROUND(sum(A.total_product_sales)/sum(B.impression_cost),2) as roas_brand, ROUND(sum(A.product_sales)/sum(B.impression_cost),2) as roas_salestracked, SUM(B.impression_cost) AS total_cost_fromconvtable From amazon_attributed_events_by_conversion_time A INNER JOIN USR_SEG_IMP B ON A.USER_ID = B.USER_ID AND A.ADVERTISER_ID = B.ADVERTISER_ID WHERE conversion_event_subtype = 'order' GROUP BY A.ADVERTISER, A.tracked_asin, B.behavior_segment_name, B.behavior_segment_matched ) SELECT A.ADVERTISER, BUILT_IN_PARAMETER('TIME_WINDOW_START') AS time_window_start, BUILT_IN_PARAMETER('TIME_WINDOW_END') AS time_window_end, A.SEGMENT, A.MATCHED_SEGMENT, A.IMPRESSIONS, A.REACH, B.tracked_asin, B.PURCHASES, B.SALES_TRACKED, B.sales_tracked_brand, A.CLICKS, roas_brand, roas_salestracked, A.impression_cost, (B.PURCHASES/A.REACH) as conversion_rate_perc, B.total_cost_fromconvtable FROM IMP_CLICKSSEG A LEFT JOIN CONV_SEG B ON A.ADVERTISER = B.ADVERTISER AND A.SEGMENT = B.SEGMENT AND A.MATCHED_SEGMENT = B.MATCHED_SEGMENT",
         "version": 1,
         "workflowId": "exposure_group_analysis_daily",
         "workflowMetaData": {
             "automaticDeployWorkflow": True,
             "endemicType": "ENDEMIC"
         },
         "workflowType": "ENDEMIC|NON-ENDEMIC"
     },
     {
         "defaultSchedule": {
             "automaticDeploySchedule": False,
             "Description": "Runs the exposure_group_analysisn workflow at 11:00 GMT / 9:00am EST weekly. ",
             "Input": {
                 "payload": {
                     "timeWindowEnd": "today(-1)",
                     "timeWindowStart": "today(-8)",
                     "timeWindowType": "EXPLICIT",
                     "workflow_executed_date": "now()"
                 }
             },
             "Name": "exposure_group_analysis_weekly,",
             "ScheduleExpression": "custom(D * 11)",
             "State": "ENABLED"
         },
         "filteredMetricsDiscriminatorColumn": "filtered",
         "sqlQuery": "WITH IMP_CLICKSSEG AS  ( SELECT A.ADVERTISER, A.behavior_segment_name AS SEGMENT, SUM(A.total_cost/100000) AS impression_cost, behavior_segment_matched AS MATCHED_SEGMENT, SUM(A.IMPRESSIONS) AS IMPRESSIONS, SUM(B.clicks) as CLICKS, COUNT(DISTINCT A.USER_ID) AS REACH FROM display_impressions_by_user_segments A LEFT JOIN DISPLAY_CLICKS B ON A.request_tag = B.request_tag GROUP BY A.ADVERTISER, A.behavior_segment_name, A.behavior_segment_matched), USR_SEG_IMP as ( select user_id, advertiser_id, behavior_segment_name, behavior_segment_matched, sum(total_cost)/100000 as impression_cost from display_impressions_by_user_segments group by user_id, advertiser_id, behavior_segment_name, behavior_segment_matched ), CONV_SEG AS ( SELECT A.ADVERTISER, A.tracked_asin, B.behavior_segment_name AS SEGMENT, B.behavior_segment_matched AS MATCHED_SEGMENT, SUM(A.CONVERSIONS) AS PURCHASES, sum(A.product_sales) as sales_tracked, sum(A.total_product_sales) as sales_tracked_brand, ROUND(sum(A.total_product_sales)/sum(B.impression_cost),2) as roas_brand, ROUND(sum(A.product_sales)/sum(B.impression_cost),2) as roas_salestracked, SUM(B.impression_cost) AS total_cost_fromconvtable From amazon_attributed_events_by_conversion_time A INNER JOIN USR_SEG_IMP B ON A.USER_ID = B.USER_ID AND A.ADVERTISER_ID = B.ADVERTISER_ID WHERE conversion_event_subtype = 'order' GROUP BY A.ADVERTISER, A.tracked_asin, B.behavior_segment_name, B.behavior_segment_matched ) SELECT A.ADVERTISER, BUILT_IN_PARAMETER('TIME_WINDOW_START') AS time_window_start, BUILT_IN_PARAMETER('TIME_WINDOW_END') AS time_window_end, A.SEGMENT, A.MATCHED_SEGMENT, A.IMPRESSIONS, A.REACH, B.tracked_asin, B.PURCHASES, B.SALES_TRACKED, B.sales_tracked_brand, A.CLICKS, roas_brand, roas_salestracked, A.impression_cost, (B.PURCHASES/A.REACH) as conversion_rate_perc, B.total_cost_fromconvtable FROM IMP_CLICKSSEG A LEFT JOIN CONV_SEG B ON A.ADVERTISER = B.ADVERTISER AND A.SEGMENT = B.SEGMENT AND A.MATCHED_SEGMENT = B.MATCHED_SEGMENT",
         "version": 1,
         "workflowId": "exposure_group_analysis_weekly",
         "workflowMetaData": {
             "automaticDeployWorkflow": True,
             "endemicType": "ENDEMIC"
         },
         "workflowType": "ENDEMIC|NON-ENDEMIC"
     }
 ]

#### Setting Multiple Workflows in Workflow Library Table


In [ ]:
workflowLibrary.set_workflow_records(workflow_list=workflow_list, TEAM_NAME=TEAM_NAME, ENV=ENV)

#### Setting Multiple Workflow Schedules in Workflows Schedule Table

In [ ]:
workflowSchedules.set_workflow_schedules(workflow_list=workflow_list, TEAM_NAME=TEAM_NAME, ENV=ENV, customer_id=customer_id)

#### Setting Multiple Workflows in Workflow Table

In [ ]:
workflows.set_workflow_records(workflow_list=workflow_list, TEAM_NAME=TEAM_NAME, ENV=ENV, customer_id=customer_id)

#### Invoke Multiple Workflows To Execute

In [ ]:
workflowInvoke.invoke_workflows(workflow_list=workflow_list, TEAM_NAME=TEAM_NAME, ENV=ENV, customer_id=customer_id)